# Test of PT-MCMC

In [1]:
var MCMC = require("../src/js/mcmc.js")
var mt = require("../../jslib/mt")
var rand = new mt(100);

var Plotly = require("ijavascript-plotly");

In [2]:
var normal = (m,s) => {
    const r1 = rand.next(), r2 = rand.next();
    const PI = Math.PI;
    return  m + s*(Math.sqrt(-2*Math.log(r1)) * Math.cos(2*PI*r2))
}

In [3]:
var normalList = Array(10000).fill(0).map(v => normal(10,1))

Plotly([
    {
        x : normalList,
        type : "histogram"
    }
])

## モデルとデータ

In [4]:
var model = (parameter, data) => {
    var m = {}
    m.x = data.x;
    m.y = data.x.map(v=>parameter[0].a*v+parameter[0].b)
    return m;
} 

var answer = (x) => x * 5 - 2;
var noise = (s) => normal(0,s);


var data = {}
data.x = Array(10).fill(0).map((v,i)=>i);
data.y = data.x.map(v=>answer(v) + noise(2));

var error = {
    x : data.x.map((v,i)=>0.),
    y : data.y.map((v,u)=>2)
}

var precise = {
    x : data.x,
    y : data.x.map(x=>answer(x))
}

console.log(data)
console.log(error)

Plotly([
    {
        x:data.x,
        y:data.y,
        mode:"markers",
        error_y: {
          type: 'data',
          array: error.y,
          visible: true
        },
    },
    {
        x:precise.x,
        y:precise.y
    }
])

{ x: [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ],
  y: 
   [ -1.9391001958215137,
     5.328804899399743,
     7.951980320334023,
     12.83714454602096,
     19.643331292905593,
     24.861139647236556,
     30.379178872936535,
     31.90924816897634,
     37.1692166823336,
     43.48471334440963 ] }
{ x: [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
  y: [ 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ] }


## パラメータ間制約なし

In [5]:
var initialParam = [{a:0,b:0}]
var updator = {
    a:{
        max:Infinity,
        min:-Infinity,
        val:0.1
    },
    b:{
        max:Infinity,
        min:-Infinity,
        val:0.1
    }
}
var constrain = {};


var mcmc = new MCMC(200).initialize(initialParam,updator, constrain)
mcmc.setObserved(data, error)
mcmc.setModel(model)
mcmc.randomizeParameters(["a","b"])

console.log(mcmc.getUpdateOrder())
console.log(mcmc.getUpdateOrder())
console.log(mcmc.getUpdateOrder())
console.log(mcmc.getUpdateOrder())
console.log(mcmc.getUpdateOrder())
console.log(mcmc.parameters)

var res = mcmc.samplingAndFormat(10000)

Plotly([
    {
        y:res.parameter[0].a
    },
    {
        y:res.parameter[0].b
    }
])


[ [ 0, 'a' ], [ 0, 'b' ] ]
[ [ 0, 'a' ], [ 0, 'b' ] ]
[ [ 0, 'a' ], [ 0, 'b' ] ]
[ [ 0, 'a' ], [ 0, 'b' ] ]
[ [ 0, 'b' ], [ 0, 'a' ] ]
[ { a: 0.13378240942137704, b: 0.2246544137327111 } ]


In [6]:
var estimatedParameter = MCMC.estimateParameter(res, 1000)
console.log(estimatedParameter)
console.log(summaryToModelParameter(estimatedParameter,"mean"))

function summaryToModelParameter(summary,useProp){
    return summary.map(s => {
        let obj = {}
        Object.entries(s).map(kv => {
            obj[kv[0]] = kv[1][useProp]
        })
        return obj
    })
}

var estimated = model(summaryToModelParameter(estimatedParameter,"mean"),data)

Plotly([
    {
        x: estimated.x,
        y: estimated.y,
        name:estimated
    },
    {
        x: precise.x,
        y: precise.y,
        name:precise
    }
])

[ { a: 
     { mean: 4.834681499117484,
       variance: 0.03494021937068369,
       stdev: 0.18692303060533683 },
    b: 
     { mean: -0.4814209393279926,
       variance: 0.9774998982633218,
       stdev: 0.9886859452138085 } } ]
[ { a: 4.834681499117484, b: -0.4814209393279926 } ]


## パラメータ間に制約あり

In [7]:
var constrain = {
    a : (cand, i, parameters) => cand > parameters[i].b,
    b : (cand, i, parameters) => cand < parameters[i].a
};


var mcmc = new MCMC(200).initialize(initialParam,updator, constrain, 0.1)
mcmc.setObserved(data, error)
mcmc.setModel(model)
mcmc.randomizeParameters(["a","b"])

console.log(mcmc.getUpdateOrder(["a","b"]))
console.log(mcmc.parameters)

var res = mcmc.samplingAndFormat(10000)

Plotly([
    {
        y:res.parameter[0].a
    },
    {
        y:res.parameter[0].b
    }
])


[ [ 0, 'a' ], [ 0, 'b' ] ]
[ { a: 0.13378240942137704, b: -0.03465813495112659 } ]


In [8]:
var estimatedParameter = MCMC.estimateParameter(res, 1000)
estimatedParameter

[ { a: 
     { mean: 4.679135748236986,
       variance: 0.16422820662551713,
       stdev: 0.40525079472533687 },
    b: 
     { mean: 0.6572007180933915,
       variance: 2.0493090786302743,
       stdev: 1.4315408057859456 } } ]

In [8]:
constrain.b(0.2815,0,[{a:0.0072,b:0.1}])

false

In [9]:
console.log(
    JSON.stringify(mcmc.sampling1set(),null," "),
    mcmc.acceptedTime,
    mcmc.sampledTime
)

[
 {
  "lnP": -316025.61748857197,
  "parameters": [
   {
    "a": 0.14719975650087183,
    "b": -0.13138479582370585
   }
  ],
  "sampled": {
   "i": 0,
   "key": "b",
   "value": -0.13138479582370585
  }
 },
 {
  "lnP": -296662.8696142427,
  "parameters": [
   {
    "a": 0.14719975650087183,
    "b": -0.13138479582370585
   }
  ],
  "sampled": {
   "i": 0,
   "key": "a",
   "value": 0.14719975650087183
  }
 }
] [ { a: 1, b: 1 } ] [ { a: 1, b: 1 } ]


In [10]:
console.log(
    JSON.stringify(mcmc.samplingAndFormat(10),null," "),
    mcmc.acceptedTime,
    mcmc.sampledTime
)


{
 "parameter": [
  {
   "a": [
    0.14719975650087183,
    0.14719975650087183,
    0.21485514906865483,
    0.21682172352143925,
    0.2614246753101702,
    0.2614246753101702,
    0.2614246753101702,
    0.2614246753101702,
    0.3008683976140879,
    0.3008683976140879
   ],
   "b": [
    -0.13138479582370585,
    -0.10793618051896037,
    -0.10793618051896037,
    -0.06059518083400736,
    -0.06059518083400736,
    0.00020725179282988365,
    0.00020725179282988365,
    0.02812001649494123,
    0.02812001649494123,
    0.06955013868821217
   ]
  }
 ],
 "lnP": [
  -296662.8696142427,
  -296662.8696142427,
  -296662.8696142427,
  -296196.89374477207,
  -287511.7396392161,
  -287511.7396392161,
  -286587.0577850462,
  -286336.9719097752,
  -280694.4908179966,
  -280694.4908179966,
  -279522.9049182144,
  -279522.9049182144,
  -279522.9049182144,
  -279522.9049182144,
  -278986.29942366446,
  -278986.29942366446,
  -274059.4719143847,
  -274059.4719143847,
  -273271.79380590364,
  -2

In [16]:
var model = (parameter, data) => {
    var m = {}
    m.x = data.x;
    m.y = data.x.map(v=>parameter[0].a*v*v+parameter[0].b*v+parameter[0].c)
    return m;
} 

var answer = (x) => x*x * 2 - 2* x + 10;
var noise = (s) => normal(0,s);


var data = {}
data.x = Array(20).fill(0).map((v,i)=>i-10);
data.y = data.x.map(v=>answer(v) + noise(2));

var error = {
    x : data.x.map((v,i)=>0.),
    y : data.y.map((v,i)=>2)
}

var precise = {
    x : data.x,
    y : data.x.map(x=>answer(x))
}

console.log(data)
console.log(error)

Plotly([
    {
        x:data.x,
        y:data.y,
        mode:"markers",
        error_y: {
          type: 'data',
          array: error.y,
          visible: true
        },
    },
    {
        x:precise.x,
        y:precise.y
    }
])

{ x: [ -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ],
  y: 
   [ 231.22839899495625,
     188.933187651271,
     151.54525003708753,
     125.20065766989792,
     96.4621492226866,
     69.38847685774695,
     49.63411995450922,
     34.09930010039437,
     20.003308371350755,
     16.897590086540298,
     11.358200865239954,
     10.476545248911883,
     13.335319620595795,
     22.714338192390343,
     33.51004822047142,
     47.62374425743154,
     73.13985588523352,
     95.10909796088484,
     122.510524469799,
     155.05478440899816 ] }
{ x: [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
  y: [ 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ] }


In [17]:
var initialParam = [{a:0,b:0,c:0}]
var updator = {
    a:{
        max:Infinity,
        min:-Infinity,
        val:0.1
    },
    b:{
        max:Infinity,
        min:-Infinity,
        val:0.1
    },
    c:{
        max:Infinity,
        min:-Infinity,
        val:0.1
    }
}
var constrain = {};


var mcmc = new MCMC(200).initialize(initialParam,updator, constrain)
mcmc.setObserved(data, error)
mcmc.setModel(model)


var res = mcmc.samplingAndFormat(10000)

Plotly([
    {
        y:res.parameter[0].a
    },
    {
        y:res.parameter[0].b
    },
    {
        y:res.parameter[0].c
    }
])


In [18]:
var estimatedParameter = MCMC.estimateParameter(res, 1000)
console.log(estimatedParameter)
console.log(summaryToModelParameter(estimatedParameter,"mean"))

function summaryToModelParameter(summary,useProp){
    return summary.map(s => {
        let obj = {}
        Object.entries(s).map(kv => {
            obj[kv[0]] = kv[1][useProp]
        })
        return obj
    })
}

var estimated = model(summaryToModelParameter(estimatedParameter,"mean"),data)

Plotly([
    {
        x: estimated.x,
        y: estimated.y,
        name : "estimated"
    },
    {
        x: precise.x,
        y: precise.y,
        name:"precise"
    }
])

[ { a: 
     { mean: 2.000182507288795,
       variance: -0.0002430154430962758,
       stdev: NaN },
    b: 
     { mean: -1.9669606895248593,
       variance: 0.006078878434782515,
       stdev: 0.07796716254156306 },
    c: 
     { mean: 10.51492223640135,
       variance: 0.34063256188092117,
       stdev: 0.5836373547682852 } } ]
[ { a: 2.000182507288795,
    b: -1.9669606895248593,
    c: 10.51492223640135 } ]
